# Algorithm 1

First allocation algorithm implemented 🚀 (Only problem: I have no clue how it works 😂) 
- only events with one hall
- only one parking lot per event (actually not 100% sure)
- only one parking lot per event (thats why fre.e + IMOT doesn't have a parking lot)
- algorithm minimizes the distance in the entire dataframe not per day (must be fixed for sure)

## Import Libraries

In [1]:
# Import Libraries
import pulp as pl
import pandas as pd
import pulp as pl

# Define Algorithm

In [2]:
def optimize_distance(df_events_parking_lot_min_capacity):

    # Define the problem
    model = pl.LpProblem("Minimize_Distance", pl.LpMinimize)

    # Define decision variables
    assignments = pl.LpVariable.dicts(
        "Assign",
        (
            (event, date, parking_lot)
            for index, (
                event,
                date,
                parking_lot,
            ) in df_events_parking_lot_min_capacity[
                ["event", "date", "parking_lot"]
            ]
            .drop_duplicates()
            .iterrows()
        ),
        cat="Binary",
    )

    # Objective: Minimize the total minimum distance
    model += pl.lpSum(
        assignments[(event, date, parking_lot)] * distance
        for index, (
            event,
            date,
            parking_lot,
            distance,
        ) in df_events_parking_lot_min_capacity[
            ["event", "date", "parking_lot", "distance"]
        ].iterrows()
    )

    # Constraint: Each event on each date should have exactly one parking lot assigned
    for (event, date), group in df_events_parking_lot_min_capacity.groupby(
        ["event", "date"]
    ):
        model += (
            pl.lpSum(
                assignments[(event, date, parking_lot)]
                for parking_lot in group["parking_lot"]
            )
            == 1
        )

    # Constraint: Do not exceed the capacity of any parking lot on any given day
    for (
        parking_lot,
        date,
    ), group in df_events_parking_lot_min_capacity.groupby(
        ["parking_lot", "date"]
    ):
        model += (
            pl.lpSum(
                assignments[(event, date, parking_lot)] * demand
                for index, (event, demand) in group[["event", "demand"]].iterrows()
            )
            <= group["capacity"].iloc[0]
        )

    # Solve the model
    model.solve()

    # Output results
    df_allocation_results = []
    for (event, date, parking_lot), var in assignments.items():
        if pl.value(var) == 1:
            df_allocation_results.append(
                {"event": event, "date": date, "parking_lot": parking_lot}
            )

    df_allocation_results = pd.DataFrame(df_allocation_results)

    # merge capacity onto df_allocation_results
    df_allocation_results = df_allocation_results.merge(
        df_events_parking_lot_min_capacity[["parking_lot"]].drop_duplicates(),
        on="parking_lot",
    )

    return df_allocation_results

## Load the data needed

In [3]:
# Load the data
df_events_parking_lot_min_capacity = pd.read_parquet('../data/output_data/df_events_parking_lot_min_capacity.parquet')
df_events_parking_lot_min_capacity.head()

,event,date,status,demand,hall,entrance,parking_lot,distance,capacity,parking_delta,max_demand
0,inhorgenta,2025-02-17,aufbau,280,A1,west,P1 Nord (Tor 17a - Tor 11c),1120,2750,2470,2576
1,inhorgenta,2025-02-17,aufbau,280,A1,west,P1 Nord (westl. Tor 17a),1070,350,70,2576
2,inhorgenta,2025-02-17,aufbau,280,A1,west,P2 Nord (östl. Tor 11c),1420,500,220,2576
3,inhorgenta,2025-02-17,aufbau,280,A1,west,P7,1320,400,120,2576
4,inhorgenta,2025-02-17,aufbau,280,A1,west,P9 - P12,1220,3000,2720,2576


## Run the optimization

In [4]:
df_allocation_results = optimize_distance(df_events_parking_lot_min_capacity)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/jakobschwarz/Documents/01_study/02_master/01_coursework/02_semester/02_Management and Digital Technologies II - Software Development Project (12)/development/southpark/venv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0x/vxtbhltd3k1g7dgrlry82kfh0000gn/T/120b580dd98c423e9498d577aeeba130-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/0x/vxtbhltd3k1g7dgrlry82kfh0000gn/T/120b580dd98c423e9498d577aeeba130-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 134 COLUMNS
At line 1050 RHS
At line 1180 BOUNDS
At line 1366 ENDATA
Problem MODEL has 129 rows, 185 columns and 360 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 28019.5 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 21 strengthened rows, 1 substitutions
Cgl0003I 0 fixed, 0 

In [5]:
df_allocation_results.head()

,event,date,parking_lot
0,inhorgenta,2025-02-17,P1 Nord (westl. Tor 17a)
1,inhorgenta,2025-02-18,P1 Nord (westl. Tor 17a)
2,inhorgenta,2025-02-19,P1 Nord (Tor 17a - Tor 11c)
3,inhorgenta,2025-02-20,P1 Nord (Tor 17a - Tor 11c)
4,inhorgenta,2025-02-21,P1 Nord (Tor 17a - Tor 11c)


## Merge Allocation results, Event data, Parking Lot distances and Parking lot capacity

In [6]:
# Load events data for merging with the results
df_events_halls = pd.read_parquet('../data/output_data/df_events_halls.parquet')

In [7]:
df_events_halls.head()

,event,date,status,demand,hall,entrance,distance_entrance
0,inhorgenta,2025-02-17,aufbau,280,A1,west,60
1,inhorgenta,2025-02-18,aufbau,330,A1,west,60
2,inhorgenta,2025-02-19,aufbau,420,A1,west,60
3,inhorgenta,2025-02-20,aufbau,420,A1,west,60
4,inhorgenta,2025-02-21,laufzeit,2278,A1,west,60


In [8]:
# load df_halls_parking_distances
df_halls_parking_distances = pd.read_parquet('../data/output_data/halls_parking_distances.parquet')

# load parking lots capacity
df_parking_lots_capacity = pd.read_csv('../data/input_data/parking_lots_capacity.csv')

# Merge the results with the events data
df_events_halls_allocated_parking_lot = pd.merge(df_events_halls, df_allocation_results, on=['event', 'date'], how='left')

# Merge capacity and distance from df_events_parking_lot_min_capacity onto df_events_halls_allocated_parking_lot
df_events_halls_allocated_parking_lot = pd.merge(
    df_events_halls_allocated_parking_lot, 
    df_events_parking_lot_min_capacity[['event', 'parking_lot', 'date', 'distance', 'capacity', 'parking_delta', 'max_demand']], 
    on=['event', 'parking_lot', 'date'], 
    how='left'
).drop_duplicates()

In [9]:
df_events_parking_lot_min_capacity.head()

,event,date,status,demand,hall,entrance,parking_lot,distance,capacity,parking_delta,max_demand
0,inhorgenta,2025-02-17,aufbau,280,A1,west,P1 Nord (Tor 17a - Tor 11c),1120,2750,2470,2576
1,inhorgenta,2025-02-17,aufbau,280,A1,west,P1 Nord (westl. Tor 17a),1070,350,70,2576
2,inhorgenta,2025-02-17,aufbau,280,A1,west,P2 Nord (östl. Tor 11c),1420,500,220,2576
3,inhorgenta,2025-02-17,aufbau,280,A1,west,P7,1320,400,120,2576
4,inhorgenta,2025-02-17,aufbau,280,A1,west,P9 - P12,1220,3000,2720,2576


In [10]:
df_events_halls_allocated_parking_lot

,event,date,status,demand,hall,entrance,distance_entrance,parking_lot,distance,capacity,parking_delta,max_demand
0,inhorgenta,2025-02-17,aufbau,280,A1,west,60,P1 Nord (westl. Tor 17a),1070.0,350.0,70.0,2576.0
1,inhorgenta,2025-02-18,aufbau,330,A1,west,60,P1 Nord (westl. Tor 17a),1070.0,350.0,20.0,2576.0
2,inhorgenta,2025-02-19,aufbau,420,A1,west,60,P1 Nord (Tor 17a - Tor 11c),1120.0,2750.0,2330.0,2576.0
3,inhorgenta,2025-02-20,aufbau,420,A1,west,60,P1 Nord (Tor 17a - Tor 11c),1120.0,2750.0,2330.0,2576.0
4,inhorgenta,2025-02-21,laufzeit,2278,A1,west,60,P1 Nord (Tor 17a - Tor 11c),1120.0,2750.0,472.0,2576.0
5,inhorgenta,2025-02-22,laufzeit,2576,A1,west,60,P9 - P12,1220.0,3000.0,424.0,2576.0
6,inhorgenta,2025-02-23,laufzeit,1795,A1,west,60,P1 Nord (Tor 17a - Tor 11c),1120.0,2750.0,955.0,2576.0
7,inhorgenta,2025-02-24,laufzeit,1828,A1,west,60,P1 Nord (Tor 17a - Tor 11c),1120.0,2750.0,922.0,2576.0
8,inhorgenta,2025-02-25,abbau,340,A1,west,60,P1 Nord (westl. Tor 17a),1070.0,350.0,10.0,2576.0
9,inhorgenta,2025-02-26,abbau,332,A1,west,60,P1 Nord (westl. Tor 17a),1070.0,350.0,18.0,2576.0


In [11]:
# First allocation algorithm results
df_events_halls_allocated_parking_lot[df_events_halls_allocated_parking_lot['date'] == pd.to_datetime('2025-02-23')][['event', 'date', 'status', 'hall', 'demand', 'parking_delta', 'capacity', 'parking_lot']].head(20)

,event,date,status,hall,demand,parking_delta,capacity,parking_lot
6,inhorgenta,2025-02-23,laufzeit,A1,1795,955.0,2750.0,P1 Nord (Tor 17a - Tor 11c)
18,Münchner Autotage,2025-02-23,laufzeit,B3,1000,2000.0,3000.0,P9 - P12
30,fre.e + IMOT,2025-02-23,laufzeit,B4,12724,NaN,NaN,NaN
33,Lopec,2025-02-23,aufbau,C6,140,360.0,500.0,P2 Nord (östl. Tor 11c)


In [12]:
# Verify, that there are no negative parking delta
if (df_events_halls_allocated_parking_lot['parking_delta'] < 0).any():
    print(df_events_halls_allocated_parking_lot[df_events_halls_allocated_parking_lot['parking_delta'] < 0])
else:
    print('No event demand exceeds allocated parking capacity')

No event demand exceeds allocated parking capacity


In [13]:
df_events_halls_allocated_parking_lot[['event', 'date', 'status', 'hall', 'demand', 'parking_delta', 'capacity', 'parking_lot', 'distance']].head(100)

,event,date,status,hall,demand,parking_delta,capacity,parking_lot,distance
0,inhorgenta,2025-02-17,aufbau,A1,280,70.0,350.0,P1 Nord (westl. Tor 17a),1070.0
1,inhorgenta,2025-02-18,aufbau,A1,330,20.0,350.0,P1 Nord (westl. Tor 17a),1070.0
2,inhorgenta,2025-02-19,aufbau,A1,420,2330.0,2750.0,P1 Nord (Tor 17a - Tor 11c),1120.0
3,inhorgenta,2025-02-20,aufbau,A1,420,2330.0,2750.0,P1 Nord (Tor 17a - Tor 11c),1120.0
4,inhorgenta,2025-02-21,laufzeit,A1,2278,472.0,2750.0,P1 Nord (Tor 17a - Tor 11c),1120.0
5,inhorgenta,2025-02-22,laufzeit,A1,2576,424.0,3000.0,P9 - P12,1220.0
6,inhorgenta,2025-02-23,laufzeit,A1,1795,955.0,2750.0,P1 Nord (Tor 17a - Tor 11c),1120.0
7,inhorgenta,2025-02-24,laufzeit,A1,1828,922.0,2750.0,P1 Nord (Tor 17a - Tor 11c),1120.0
8,inhorgenta,2025-02-25,abbau,A1,340,10.0,350.0,P1 Nord (westl. Tor 17a),1070.0
9,inhorgenta,2025-02-26,abbau,A1,332,18.0,350.0,P1 Nord (westl. Tor 17a),1070.0


In [14]:
# Check for duplicate parking lot allocations
# Group by 'date' and 'parking_lot' and count occurrences
parking_lot_occurrences = df_events_halls_allocated_parking_lot.groupby(['date', 'parking_lot']).size()

# Filter to find any date-parking lot combinations with more than one occurrence
duplicate_parking_lots = parking_lot_occurrences[parking_lot_occurrences > 1]

# Display the results
print(duplicate_parking_lots)


date        parking_lot                
2025-02-16  P2 Nord (östl. Tor 11c)        2
2025-02-19  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-20  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-24  P2 Nord (östl. Tor 11c)        3
2025-02-25  P2 Nord (östl. Tor 11c)        3
dtype: int64


# Algorithm 2 (WIP)

This currently assigns only one parking lot per event but it also assigns the same parking lot to multiple events

In [15]:
def optimize_distance(df_events_parking_lot_min_capacity):
    # Filter out rows where parking lot capacity is less than the max demand for the event
    feasible_allocations = df_events_parking_lot_min_capacity[
        df_events_parking_lot_min_capacity['capacity'] >= df_events_parking_lot_min_capacity['max_demand']
    ]

    # Define the problem
    model = pl.LpProblem("Minimize_Distance", pl.LpMinimize)

    # Define decision variables for feasible allocations only
    assignments = pl.LpVariable.dicts(
        "Assign",
        (
            (event, parking_lot)
            for event, parking_lot in feasible_allocations[['event', 'parking_lot']].drop_duplicates().itertuples(index=False)
        ),
        cat="Binary"
    )

    # Objective: Minimize the total minimum distance
    model += pl.lpSum(
        assignments[(event, parking_lot)] * pl.lpSum(
            distance for _, (ev, dt, pl, distance) in 
            feasible_allocations[feasible_allocations['event'] == event][['event', 'date', 'parking_lot', 'distance']].iterrows() if pl == parking_lot
        )
        for event, parking_lot in assignments
    )

    # Constraint: Each event should have exactly one parking lot assigned
    for event in feasible_allocations['event'].unique():
        model += pl.lpSum(
            assignments[(event, parking_lot)] for parking_lot in 
            feasible_allocations[feasible_allocations['event'] == event]['parking_lot'].unique()
        ) == 1

    # Solve the model
    model.solve()

    # Output results
    df_allocation_results = []
    for (event, parking_lot), var in assignments.items():
        if pl.value(var) == 1:
            event_dates = feasible_allocations[
                (feasible_allocations['event'] == event) & 
                (feasible_allocations['parking_lot'] == parking_lot)
            ]['date'].unique()
            for date in event_dates:
                df_allocation_results.append({
                    "event": event,
                    "date": date,
                    "parking_lot": parking_lot
                })


    df_allocation_results = pd.DataFrame(df_allocation_results)

    return df_allocation_results


## Load the data needed

In [16]:
# Load the data again
df_events_parking_lot_min_capacity = pd.read_parquet('../data/output_data/df_events_parking_lot_min_capacity.parquet')

In [17]:
df_events_parking_lot_min_capacity.head()

,event,date,status,demand,hall,entrance,parking_lot,distance,capacity,parking_delta,max_demand
0,inhorgenta,2025-02-17,aufbau,280,A1,west,P1 Nord (Tor 17a - Tor 11c),1120,2750,2470,2576
1,inhorgenta,2025-02-17,aufbau,280,A1,west,P1 Nord (westl. Tor 17a),1070,350,70,2576
2,inhorgenta,2025-02-17,aufbau,280,A1,west,P2 Nord (östl. Tor 11c),1420,500,220,2576
3,inhorgenta,2025-02-17,aufbau,280,A1,west,P7,1320,400,120,2576
4,inhorgenta,2025-02-17,aufbau,280,A1,west,P9 - P12,1220,3000,2720,2576


## Run the optimization

In [18]:
# Run the new allocation algorithm
df_allocation_results_2 = optimize_distance(df_events_parking_lot_min_capacity)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/jakobschwarz/Documents/01_study/02_master/01_coursework/02_semester/02_Management and Digital Technologies II - Software Development Project (12)/development/southpark/venv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0x/vxtbhltd3k1g7dgrlry82kfh0000gn/T/087362fc0da2442fbec26ca82c290653-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/0x/vxtbhltd3k1g7dgrlry82kfh0000gn/T/087362fc0da2442fbec26ca82c290653-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 54 RHS
At line 59 BOUNDS
At line 71 ENDATA
Problem MODEL has 4 rows, 11 columns and 11 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 32120 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No inte

## Merge Allocation results, Event data, Parking Lot distances and Parking lot capacity

In [19]:
# load df_halls_parking_distances
df_halls_parking_distances = pd.read_parquet('../data/output_data/halls_parking_distances.parquet')

# load parking lots capacity
df_parking_lots_capacity = pd.read_csv('../data/input_data/parking_lots_capacity.csv')

# Merge the results with the events data
df_events_halls_allocated_parking_lot = pd.merge(df_events_halls, df_allocation_results_2, on=['event', 'date'], how='left')

# Merge capacity and distance from df_events_parking_lot_min_capacity onto df_events_halls_allocated_parking_lot
df_events_halls_allocated_parking_lot = pd.merge(
    df_events_halls_allocated_parking_lot, 
    df_events_parking_lot_min_capacity[['event', 'parking_lot', 'date', 'distance', 'capacity', 'parking_delta', 'max_demand']], 
    on=['event', 'parking_lot', 'date'], 
    how='left'
).drop_duplicates()

## Output

In [20]:
# Alternative allocation algorithm results does not solve the problem
pd.set_option('display.max_rows', None)
df_events_halls_allocated_parking_lot[['event', 'date', 'status', 'hall', 'demand', 'parking_delta', 'capacity', 'parking_lot', 'distance', 'max_demand']].head(100).reset_index(drop=True)

,event,date,status,hall,demand,parking_delta,capacity,parking_lot,distance,max_demand
0,inhorgenta,2025-02-17,aufbau,A1,280,2470.0,2750.0,P1 Nord (Tor 17a - Tor 11c),1120.0,2576.0
1,inhorgenta,2025-02-18,aufbau,A1,330,2420.0,2750.0,P1 Nord (Tor 17a - Tor 11c),1120.0,2576.0
2,inhorgenta,2025-02-19,aufbau,A1,420,2330.0,2750.0,P1 Nord (Tor 17a - Tor 11c),1120.0,2576.0
3,inhorgenta,2025-02-20,aufbau,A1,420,2330.0,2750.0,P1 Nord (Tor 17a - Tor 11c),1120.0,2576.0
4,inhorgenta,2025-02-21,laufzeit,A1,2278,472.0,2750.0,P1 Nord (Tor 17a - Tor 11c),1120.0,2576.0
5,inhorgenta,2025-02-22,laufzeit,A1,2576,174.0,2750.0,P1 Nord (Tor 17a - Tor 11c),1120.0,2576.0
6,inhorgenta,2025-02-23,laufzeit,A1,1795,955.0,2750.0,P1 Nord (Tor 17a - Tor 11c),1120.0,2576.0
7,inhorgenta,2025-02-24,laufzeit,A1,1828,922.0,2750.0,P1 Nord (Tor 17a - Tor 11c),1120.0,2576.0
8,inhorgenta,2025-02-25,abbau,A1,340,2410.0,2750.0,P1 Nord (Tor 17a - Tor 11c),1120.0,2576.0
9,inhorgenta,2025-02-26,abbau,A1,332,2418.0,2750.0,P1 Nord (Tor 17a - Tor 11c),1120.0,2576.0


In [21]:
# Check for duplicate parking lot allocations
# Group by 'date' and 'parking_lot' and count occurrences
parking_lot_occurrences = df_events_halls_allocated_parking_lot.groupby(['date', 'parking_lot']).size()

# Filter to find any date-parking lot combinations with more than one occurrence
duplicate_parking_lots = parking_lot_occurrences[parking_lot_occurrences > 1]

# Display the results
print(duplicate_parking_lots)


date        parking_lot                
2025-02-17  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-18  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-19  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-20  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-21  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-22  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-23  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-24  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-25  P1 Nord (Tor 17a - Tor 11c)    2
dtype: int64
